In [12]:
import databento as db
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import os


In [13]:
API_KEY = os.environ['DATABENTO_KEY']
DATASET = 'XNAS.ITCH'  # NASDAQ ITCH data or other supported dataset
SYMBOL = 'AAPL'  # Change this to your desired stock symbol
START_DATE = '2018-05-01'  # Change to the earliest date allowed by your subscription
END_DATE = '2025-02-28'  # Or the most recent date


In [14]:
client = db.Historical(API_KEY)

In [15]:
print(client.metadata.list_datasets())

['ARCX.PILLAR', 'DBEQ.BASIC', 'EPRL.DOM', 'EQUS.MINI', 'EQUS.SUMMARY', 'GLBX.MDP3', 'IEXG.TOPS', 'IFEU.IMPACT', 'MEMX.MEMOIR', 'NDEX.IMPACT', 'OPRA.PILLAR', 'XASE.PILLAR', 'XBOS.ITCH', 'XCHI.PILLAR', 'XCIS.TRADESBBO', 'XNAS.BASIC', 'XNAS.ITCH', 'XNYS.PILLAR', 'XPSX.ITCH']


In [16]:
from pathlib import Path

# Jupyter notebook runs in the current working directory
current_folder = Path.cwd()
print(current_folder)
subfolder = current_folder / "data"
print(subfolder)
history_file = subfolder / "AAPL_minute_data.csv"
got_ticker = False

if history_file.exists():
    print("File exists!")
    got_ticker = True
else:
    print("File does not exist.")


/Users/carloteufel/Documents/Projects/FeatureEngineering
/Users/carloteufel/Documents/Projects/FeatureEngineering/data
File exists!


In [17]:
if got_ticker:
    print("Reading file...")
    df = pd.read_csv(history_file)
    print(df.head())
else:
    print("Fetching data...")
    # Fetch minute-bar data
    df = client.timeseries.get_range(
        dataset=DATASET,
        symbols=SYMBOL,
        schema='ohlcv-1m',
        start=START_DATE,
        end=END_DATE,
    ).to_df()

    # Display the dat
    # Optionally, save data to a CSV file
    df.to_csv(f'data/{SYMBOL}_minute_data.csv')


Reading file...
                    ts_event  rtype  publisher_id  instrument_id    open  \
0  2018-05-01 08:27:00+00:00     33             2             13  166.40   
1  2018-05-01 08:35:00+00:00     33             2             13  166.39   
2  2018-05-01 08:38:00+00:00     33             2             13  166.10   
3  2018-05-01 08:39:00+00:00     33             2             13  166.20   
4  2018-05-01 08:45:00+00:00     33             2             13  166.00   

     high     low   close  volume symbol  
0  166.40  166.40  166.40       3   AAPL  
1  166.39  166.39  166.39       2   AAPL  
2  166.10  166.05  166.05    1400   AAPL  
3  166.20  166.20  166.20      23   AAPL  
4  166.00  166.00  166.00     100   AAPL  


In [18]:
df.columns

Index(['ts_event', 'rtype', 'publisher_id', 'instrument_id', 'open', 'high',
       'low', 'close', 'volume', 'symbol'],
      dtype='object')

In [19]:


df2 = df.copy()

time_counts = df2.groupby(df2.index.time)['volume'].sum()

time_counts.index = time_counts.index.astype(str)


fig = go.Figure()

# Add the main line trace
fig.add_trace(go.Line(x=time_counts.index, y=time_counts.values, mode='lines', name='Sum of Values'))

# Add markers at the detected peaks


AttributeError: 'RangeIndex' object has no attribute 'time'